# Rocket Lab ML Models
This notebook trains and registers the ML models used by the Rocket Lab Intelligence Agent.

**Models:**
1. `MISSION_RISK_PREDICTOR`: Predicts mission success/failure risk.
2. `SUPPLIER_QUALITY_PREDICTOR`: Predicts supplier quality issues.
3. `COMPONENT_FAILURE_PREDICTOR`: Predicts component failure likelihood.

**Data Source:** Uses Feature Views defined in `04_create_views.sql` (Single Source of Truth).

In [ ]:
from snowflake.snowpark import Session
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import pandas as pd

# Connect to Snowflake
# (Assuming session is created via connection parameters or integrated env)
# session = Session.builder.configs({...}).create()
print(f"Current Database: {session.get_current_database()}")
print(f"Current Schema: {session.get_current_schema()}")

## 1. Mission Risk Predictor

In [ ]:
# Load training data from Feature View
mission_df = session.table("ROCKET_LAB_INTELLIGENCE.ANALYTICS.V_MISSION_RISK_FEATURES")

# Define features and label (MUST MATCH VIEW COLUMNS EXACTLY)
mission_features = ['WEATHER_RISK', 'TECHNICAL_RISK', 'PAYLOAD_MASS', 'CONTRACT_VAL']
mission_label = 'RISK_LABEL'

# Train Model
rf_mission = RandomForestClassifier(
    input_cols=mission_features,
    label_cols=mission_label,
    n_estimators=100,
    max_depth=5
)

rf_mission.fit(mission_df)

# Register Model
reg = Registry(session=session, database_name="ROCKET_LAB_INTELLIGENCE", schema_name="ANALYTICS")

model_ref_mission = reg.log_model(
    model_name="MISSION_RISK_PREDICTOR",
    model_version=None, # Auto-versioning
    model=rf_mission,
    sample_input_data=mission_df.select(mission_features).limit(10)
)

print("MISSION_RISK_PREDICTOR registered successfully.")

## 2. Supplier Quality Predictor

In [ ]:
# Load data
supplier_df = session.table("ROCKET_LAB_INTELLIGENCE.ANALYTICS.V_SUPPLIER_QUALITY_FEATURES")

# Features
sup_features = ['QUALITY_SCORE', 'DELIVERY_SCORE', 'RISK_METRIC', 'SPEND_AMOUNT']
sup_label = 'QUALITY_LABEL'

# Train
rf_sup = RandomForestClassifier(
    input_cols=sup_features,
    label_cols=sup_label,
    n_estimators=50
)

rf_sup.fit(supplier_df)

# Register
model_ref_sup = reg.log_model(
    model_name="SUPPLIER_QUALITY_PREDICTOR",
    model_version=None,
    model=rf_sup,
    sample_input_data=supplier_df.select(sup_features).limit(10)
)

print("SUPPLIER_QUALITY_PREDICTOR registered successfully.")

## 3. Component Failure Predictor

In [ ]:
# Load data
comp_df = session.table("ROCKET_LAB_INTELLIGENCE.ANALYTICS.V_COMPONENT_FAILURE_FEATURES")

# Features
comp_features = ['CYCLE_COUNT', 'AGE_DAYS']
comp_label = 'FAILURE_LABEL'

# Train
rf_comp = RandomForestClassifier(
    input_cols=comp_features,
    label_cols=comp_label,
    n_estimators=50
)

rf_comp.fit(comp_df)

# Register
model_ref_comp = reg.log_model(
    model_name="COMPONENT_FAILURE_PREDICTOR",
    model_version=None,
    model=rf_comp,
    sample_input_data=comp_df.select(comp_features).limit(10)
)

print("COMPONENT_FAILURE_PREDICTOR registered successfully.")

In [ ]:
# Verify Models
models = reg.show_models()
print(models)